In [48]:
import os
import time
import sys
import h5py
import re
import tensorflow  as tf
import numpy       as np
import pandas      as pd
from matplotlib    import pyplot as plt 
from os            import path
from tqdm          import tqdm

print("="*50) 
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.5.0
 Eager execution: True


In [49]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
ExcitType           = 'KInel'
NNRunIdx            = 12

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures/' + ExcitType + '/' 

TTranVec            = [1500.0]
Molecules           = ['O2','O2']
Atoms               = ['O','O']

In [50]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata, read_groupsmapping, read_sampledinitiallevels

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...
/home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/


In [51]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 
InputData.KineticFldr     = WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run'+str(NNRunIdx)+'/'
InputData.QCTKineticFldr  = WORKSPACE_PATH+'/Air_Database/Run_0D/database/kinetics/O3_UMN/'


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [52]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg  = 0
InputData.TrainIntFlg        = 0
InputData.TransferFlg        = False
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'rMin_i', 'rMax_i', 'VMin_i', 'VMax_i', 'Tau_i', 'ri_i', 'ro_i', 'TTran_i', 'EVib_Delta', 'ERot_Delta', 'rMin_Delta', 'rMax_Delta', 'VMin_Delta', 'VMax_Delta', 'Tau_Delta', 'ri_Delta', 'ro_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/Params//ModelCheckpoint/cp-8534.ckpt


In [53]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
MinValueTrain      = 1.e-16 * InputData.MultFact
MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

[SurQCT]:   Reading Variables:  ['Tau', 'VMin', 'EVib', 'VMax', 'vqn', 'jqn', 'ERot', 'ro', 'rMin', 'rMax', 'ri']


In [54]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array([TTranVec]), np.array([TTranVec]))
    DiatData.append(DiatDataTemp)
    
    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)
    
GroupMap = read_groupsmapping(InputData.PathToGrouping)

[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:       Reading Grouping from File: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv


In [55]:
#========================================================================================================
def read_kexcitdata_PLATO(InputData, PathToKineticFldr, TTra):
    print(PathToKineticFldr)
    PathToFile    = PathToKineticFldr + '/T' + str(int(TTra)) + 'K/Inel.dat'
    Rates = np.zeros((NLevels[0],NLevels[0]))
    i2j_dict = {}
    with open(PathToFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('O2|O|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            jIdx    = int(linestemp[8]);
            RateTemp = float(linestemp[12]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates, i2j_dict

#========================================================================================================
def write_predictiondata(KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):

    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
            csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


Reading QCT Kinetics used in Plato

In [56]:
QCTRates, i2j_dict = read_kexcitdata_PLATO(InputData,InputData.QCTKineticFldr,TTranVec[0])

/home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/kinetics/O3_UMN/


In [58]:
for TTran in TTranVec:
    ### Opening Files                                                                                                                                                                 
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):
        KineticFile_KInel = PathToKineticFldr + '/Inel_QCTTransitions.dat'
        csvkinetics_KInel = write_predictiondata(KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1.dat'
        csvkinetics_KExch = write_predictiondata(KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)
            
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(i2j_dict.keys(), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.02)
        
        jIdxVec  = i2j_dict[iIdx]
        jNLevels = len(jIdxVec)
    
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
    
        TTranVec              = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
        TTranDataTemp.index   = jIdxVec

    
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
    
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

    
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
    
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
    
   
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
        
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)
                           
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)
    

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 1/297 [00:00<00:33,  8.86it/s]

[SurQCT]:   Writing Kinetics in File: /home/maitreyee/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run12//T1500K/Inel_QCTTransitions.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 297/297 [00:19<00:00, 15.43it/s]

[SurQCT]:   Closing Kinetics File: /home/maitreyee/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run12//T1500K/Inel_QCTTransitions.dat
